In [ ]:
import carla
import numpy as np
import time
import pyproj

# Connect to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

# Load the world and choose a vehicle
world = client.get_world()
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
spawn_point = world.get_map().get_spawn_points()[75]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)


In [2]:
# Initialize pyproj for coordinate transformation (GPS to ENU)
proj = pyproj.Proj(proj="latlong", datum="WGS84")  # Latitude, Longitude, Altitude
enu_proj = pyproj.Proj(proj="utm", zone=33, datum="WGS84")  # UTM projection (adjust if necessary)

In [ ]:
 vehicle.destroy()

In [ ]:
# import pyproj
# # Reference GPS point (latitude, longitude, altitude) at the starting location
# ref_lat = 0.0  # Replace with the actual latitude of your reference point
# ref_lon = 0.0  # Replace with the actual longitude of your reference point
# ref_alt = 0.0  # Replace with the actual altitude of your reference point

# # Set up the pyproj transformer (from geographic to ENU)
# geod = pyproj.Geod(ellps='WGS84')
# proj = pyproj.Proj(proj="latlong", datum="WGS84")

In [3]:
# Function to convert GPS (lat, lon, alt) to ENU coordinates
def gps_to_enu(lat, lon, alt):
    east, north, up = pyproj.transform(proj, enu_proj, lon, lat, alt)
#     print(f"ENU Coordinates: East={east}, North={north}, Up={up}")  # Debugging output
    return east, north, up



In [4]:
# Add sensors
gps_bp = blueprint_library.find('sensor.other.gnss')
imu_bp = blueprint_library.find('sensor.other.imu')
lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')

gps = world.spawn_actor(gps_bp, carla.Transform(), attach_to=vehicle)
imu = world.spawn_actor(imu_bp, carla.Transform(), attach_to=vehicle)
lidar = world.spawn_actor(lidar_bp, carla.Transform(carla.Location(z=2.0)), attach_to=vehicle)

In [5]:
# Variables to store data
gps_data = None
imu_data = (0, 0, 0)
lidar_data = []

In [6]:
# def gps_to_enu(lat, lon, alt):
#     # Transform GPS (lat, lon, alt) to ENU coordinates
#     east, north, up = pyproj.transform(proj, enu_proj, lon, lat, alt)
#     return east, north, up

In [7]:
# Sensor callbacks
def gps_callback(data):
    global gps_data
    gps_data = (data.latitude, data.longitude, data.altitude)
#     print(f"GPS Data: {gps_data}")  # Print out GPS data for debugging
def imu_callback(data):
    global imu_data
    imu_data = (data.accelerometer.x, data.accelerometer.y, data.accelerometer.z)

def lidar_callback(data):
    global lidar_data
    points = np.frombuffer(data.raw_data, dtype=np.float32).reshape(-1, 4)
    lidar_data = points

In [8]:
# Attach sensors to listen for data
gps.listen(gps_callback)
imu.listen(imu_callback)
lidar.listen(lidar_callback)

In [9]:
# Initial state and covariance matrix (initialize in ENU coordinates)
# Wait for GPS data to initialize the state
while gps_data is None:
    time.sleep(0.1)  # Wait until we get the GPS data

ref_lat, ref_lon, ref_alt = gps_data  # Use the GPS data for reference

# Convert the GPS coordinates (lat, lon, alt) to ENU coordinates
initial_enu_coords = gps_to_enu(ref_lat, ref_lon, ref_alt)
state = np.array([initial_enu_coords[0], initial_enu_coords[1], initial_enu_coords[2], 0, 0, 0])  # [x, y, z, vx, vy, vz]
P = np.eye(6) * 0.02  # Initial covariance matrix with small uncertainties

# Define process noise and measurement noise (tunable parameters)
Q = np.diag([0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001])  # Process noise covariance
R_gps = np.diag([0.05, 0.05, 0.05])  # GPS measurement noise
R_lidar = np.diag([0.3, 0.3])  # LiDAR measurement noise

C:\Users\Haneen Soliman\.conda\envs\carla-sim\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
# EKF Prediction Step
def predict(state, P, imu_data, dt):
    ax, ay, az = imu_data

    # State transition matrix
    F = np.eye(6)
    F[0, 3] = F[1, 4] = F[2, 5] = dt

    # Control input based on IMU accelerations
    u = np.array([0, 0, 0, ax, ay, az])

    # Predict the state and update the covariance
    state = F @ state + u * dt
    P = F @ P @ F.T + Q  # Update the covariance with process noise

    return state, P

# GPS Update Step
def update_with_gps(state, P, gps_data):
    lat, lon, alt = gps_data
    # Convert GPS data to ENU coordinates
    enu_coords = gps_to_enu(lat, lon, alt)
    z = np.array([enu_coords[0], enu_coords[1], enu_coords[2]])  # GPS measurement in ENU

    # Measurement matrix
    H = np.array([
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0]
    ])

    # Measurement residual
    y = z - H @ state
    S = H @ P @ H.T + R_gps
    K = P @ H.T @ np.linalg.inv(S)  # Kalman gain

    # Update state and covariance
    state = state + K @ y
    P = (np.eye(6) - K @ H) @ P

    return state, P

# LiDAR Update Step
def update_with_lidar(state, P, lidar_data):
    if len(lidar_data) > 0:
        # Estimate distance from the number of LiDAR points
        distance = len(lidar_data) * 0.01
        z = np.array([
            state[0] + distance * np.cos(state[2]),  # LiDAR x update
            state[1] + distance * np.sin(state[2])   # LiDAR y update
        ])

        # Measurement matrix for LiDAR
        H = np.array([
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0]
        ])

        # Measurement residual
        y = z - H @ state
        S = H @ P @ H.T + R_lidar
        K = P @ H.T @ np.linalg.inv(S)  # Kalman gain

        # Update state and covariance
        state = state + K @ y
        P = (np.eye(6) - K @ H) @ P

    return state, P

In [11]:
# Main loop
try:
    vehicle.set_autopilot(True)  # Enable autopilot once outside the loop

    while True:
        loop_start = time.time()  # Start time for each loop iteration
        dt = 0.1  # Time step (100 ms)

        # EKF prediction and updates if data is available
        if gps_data is not None and imu_data is not None and len(lidar_data) > 0:
            state, P = predict(state, P, imu_data, dt)  # Prediction step
            state, P = update_with_gps(state, P, gps_data)  # GPS update
            state, P = update_with_lidar(state, P, lidar_data)  # LiDAR update

        # Print the current estimated state (every 10 seconds for clarity)
        if int(time.time()) % 10 == 0:
            print("Current state:", state)
            print("GPS Data:", gps_data)

        # Sleep to maintain a consistent loop cycle time
        elapsed_time = time.time() - loop_start
        time.sleep(max(0, dt - elapsed_time))  # Ensure stable loop time

except KeyboardInterrupt:
    print("Simulation stopped by user.")
finally:
    # Clean up resources
    vehicle.destroy()
    gps.destroy()
    imu.destroy()
    lidar.destroy()

C:\Users\Haneen Soliman\.conda\envs\carla-sim\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  This is separate from the ipykernel package so we can avoid doing imports until


Current state: [-1.18854019e+06  2.07481964e+02  8.96776585e+00  1.28060097e+00
  9.61891787e-02  1.66737858e+01]
GPS Data: (0.0018121601440554969, 0.0009929265143682117, 0.0016006279038265347)
Current state: [-1.18854071e+06  2.07488756e+02  9.34940815e+00  9.66442395e-01
  9.65866293e-02  1.70197872e+01]
GPS Data: (0.0018122815898777844, 0.0009880998653597189, 0.0016052436549216509)
Current state: [-1.18854125e+06  2.07474939e+02  9.73482952e+00  6.55985665e-01
  8.91715494e-02  1.73760335e+01]
GPS Data: (0.0018123854904956715, 0.0009839480219761834, 0.0015686225378885865)
Current state: [-1.18854180e+06  2.07442585e+02  1.01246976e+01  3.59775542e-01
  7.32315986e-02  1.77271244e+01]
GPS Data: (0.0018125155718422548, 0.0009787605993733604, 0.0015701103257015347)
Current state: [-1.18854234e+06  2.07401892e+02  1.05181681e+01  8.97292777e-02
  3.42246253e-02  1.80569223e+01]
GPS Data: (0.0018126412669090541, 0.0009737553455041486, 0.0016132545424625278)
Current state: [-1.18854290e+0

Current state: [-1.18866930e+06  2.10810578e+02  2.07867078e+01 -5.53046055e+00
  1.30459437e-01  2.29515757e+01]
GPS Data: (0.00183925243162264, -8.459096469482365e-05, 0.0015006065368652344)
Current state: [-1.18871502e+06  2.11916408e+02  2.07179461e+01 -5.52417541e+00
 -3.07130929e-01  2.28353189e+01]
GPS Data: (0.0018476346606774996, -0.00048180617882227734, 0.0015094184782356024)
Current state: [-1.18871558e+06  2.11881052e+02  2.07166820e+01 -5.52532792e+00
 -3.73934736e-01  2.28475615e+01]
GPS Data: (0.0018472758060852357, -0.0004864824255411126, 0.0013409614330157638)
Current state: [-1.18871613e+06  2.11841669e+02  2.07166862e+01 -5.52783156e+00
 -4.63872610e-01  2.28361238e+01]
GPS Data: (0.001846863493412343, -0.0004909803415432975, 0.0017452811589464545)
Current state: [-1.18871668e+06  2.11792576e+02  2.07156332e+01 -5.52098217e+00
 -5.01502691e-01  2.29048819e+01]
GPS Data: (0.0018463840154367972, -0.0004954642419299732, 0.0014768409309908748)
Current state: [-1.18871723

Current state: [-1.18874712e+06  5.41710060e+01  2.02481888e+01  3.27270783e-01
 -5.43190475e+00  2.31171215e+01]
GPS Data: (0.0004707824688665596, -0.0007632384017950708, -0.6101190447807312)
Current state: [-1.18874711e+06  5.36296396e+01  2.02589798e+01  3.31020959e-01
 -5.43099847e+00  2.31189517e+01]
GPS Data: (0.0004661397705234549, -0.0007631891929347498, -0.6025605797767639)
Current state: [-1.18874709e+06  5.30996462e+01  2.02695992e+01  3.38675601e-01
 -5.43307981e+00  2.30960582e+01]
GPS Data: (0.0004616049479011508, -0.0007631419716189266, -0.5951575040817261)
Current state: [-1.18874708e+06  5.25625107e+01  2.02779403e+01  3.45871152e-01
 -5.42621653e+00  2.30895900e+01]
GPS Data: (0.0004564675908369509, -0.0007630896786378335, -0.586715579032898)
Current state: [-1.18874707e+06  5.20333974e+01  2.02855999e+01  3.53838126e-01
 -5.42211473e+00  2.31411630e+01]
GPS Data: (0.0004519754661345132, -0.0007630449246186281, -0.5793702006340027)
Current state: [-1.18874999e+06  1.2

Current state: [-1.18874408e+06  9.87484491e+00  2.07097600e+01  3.16758648e+00
 -2.10479337e+00  2.29707127e+01]
GPS Data: (5.402013155730856e-05, -0.0007630272423262008, -0.03453189879655838)
Current state: [-1.18874409e+06  9.32774216e+00  2.07181744e+01  3.02489430e+00
 -2.25996962e+00  2.29798780e+01]
GPS Data: (4.9411804042165386e-05, -0.0007630194292202446, -0.0321710966527462)
Current state: [-1.18874412e+06  8.77754840e+00  2.07268410e+01  2.88489285e+00
 -2.40534742e+00  2.29550863e+01]
GPS Data: (4.483033833935224e-05, -0.0007630107251460652, -0.029526786878705025)
Current state: [-1.18874416e+06  8.21044515e+00  2.07326605e+01  2.74514627e+00
 -2.53599020e+00  2.29780720e+01]
GPS Data: (4.006882424789637e-05, -0.0007629992796311995, -0.027044372633099556)
Current state: [-1.18874421e+06  7.62822812e+00  2.07402620e+01  2.60904006e+00
 -2.67595092e+00  2.29801850e+01]
GPS Data: (3.46317720669731e-05, -0.0007629855724277675, -0.02412647195160389)
Current state: [-1.18874426e+

Current state: [-1.18874721e+06 -1.04790581e+02  2.03428489e+01 -2.88785324e-01
  9.17343102e-02  2.20783315e+01]
GPS Data: (-0.0009192837414673249, -0.0007596491705764017, 0.32530122995376587)
Current state: [-1.18874721e+06 -1.04790585e+02  2.03428490e+01 -2.88785620e-01
  9.17315803e-02  2.20783316e+01]
GPS Data: (-0.0009192837414673249, -0.0007596491705764017, 0.32530122995376587)
Current state: [-1.18874721e+06 -1.04790589e+02  2.03428491e+01 -2.88785880e-01
  9.17290086e-02  2.20783317e+01]
GPS Data: (-0.0009192837414673249, -0.0007596491705764017, 0.32530122995376587)
Current state: [-1.18874721e+06 -1.04790592e+02  2.03428492e+01 -2.88786106e-01
  9.17265911e-02  2.20783317e+01]
GPS Data: (-0.0009192837414673249, -0.0007596491705764017, 0.32530122995376587)
Current state: [-1.18874721e+06 -1.04790596e+02  2.03428493e+01 -2.88786301e-01
  9.17243235e-02  2.20783318e+01]
GPS Data: (-0.0009192837414673249, -0.0007596491705764017, 0.32530122995376587)
Current state: [-1.18874721e+0